In [94]:
from itertools import combinations
from math import comb, factorial
from random import shuffle

import numpy as np
import pandas as pd

In [68]:
teams = list('ABCDEFGH')
list(combinations(teams, 2))

[('A', 'B'),
 ('A', 'C'),
 ('A', 'D'),
 ('A', 'E'),
 ('A', 'F'),
 ('A', 'G'),
 ('A', 'H'),
 ('B', 'C'),
 ('B', 'D'),
 ('B', 'E'),
 ('B', 'F'),
 ('B', 'G'),
 ('B', 'H'),
 ('C', 'D'),
 ('C', 'E'),
 ('C', 'F'),
 ('C', 'G'),
 ('C', 'H'),
 ('D', 'E'),
 ('D', 'F'),
 ('D', 'G'),
 ('D', 'H'),
 ('E', 'F'),
 ('E', 'G'),
 ('E', 'H'),
 ('F', 'G'),
 ('F', 'H'),
 ('G', 'H')]

In [95]:
factorial(comb(8, 2))

304888344611713860501504000000

In [113]:
def get_matches(teams):
    matches = list(combinations(teams, 2))
#     shuffle(matches)
    return matches

In [106]:
def shares_team(match_1, match_2):
    return len(set(match_1).intersection(set(match_2))) > 0

def any_back_to_back(matches):
    return any([shares_team(*match) for match in pairwise(matches)])

In [109]:
def get_team_priorities(teams, played):
    priorities = {}
    for distance, match in enumerate(played[::-1]):
        for team in match:
            if team not in priorities:
                priorities[team] = distance
    return priorities


def get_match_priorities(teams, scheduled, remaining):
    priorities = []
    unscheduled_priority = len(teams)
    team_priorities = get_team_priorities(teams, scheduled)
    for team_1, team_2 in remaining:
        team_1_priority = team_priorities.get(team_1, unscheduled_priority)
        team_2_priority = team_priorities.get(team_2, unscheduled_priority)
        priorities.append(team_1_priority * team_2_priority)
    return priorities

In [134]:
def get_game_intervals(matches):
    teams = sorted(set([team for match in matches for team in match]))
    intervals = {}
    for team in teams:
        team_intervals = []
        last_idx = -1
        for idx, match in enumerate(matches):
            if team in match:
                team_intervals.append(idx - last_idx - 1)
                last_idx = idx
        team_intervals.append(idx - last_idx)
        intervals[team] = team_intervals
    return intervals

In [136]:
def get_priority_schedule(teams, num_candidates=5):
    all_matches = get_matches(teams)
    
    def get_candidate_schedule():
        scheduled = []
        remaining = all_matches[:]
        while len(scheduled) < len(all_matches):
            priorities = get_match_priorities(teams, scheduled, remaining)
            max_priority = max(priorities)
            highest = [idx for idx, priority in enumerate(priorities) if priority == max_priority]
            shuffle(highest)
            scheduled.append(remaining.pop(highest[0]))
        return scheduled
    
    def score_schedule(schedule):
        game_intervals = get_game_intervals(schedule)
        return np.var([i for intvls in game_intervals.values() for i in intvls])
    
    candidate_schedules = [get_candidate_schedule() for _ in range(num_candidates)]
    candidate_scores = [score_schedule(candidate) for candidate in candidate_schedules]
    best_idx = np.argmin(candidate_scores)
    return candidate_schedules[best_idx]

In [121]:
get_priority_schedule(teams[:6])

[('C', 'F'),
 ('A', 'D'),
 ('B', 'E'),
 ('C', 'D'),
 ('A', 'F'),
 ('C', 'E'),
 ('B', 'D'),
 ('A', 'E'),
 ('B', 'F'),
 ('A', 'C'),
 ('D', 'E'),
 ('B', 'C'),
 ('E', 'F'),
 ('A', 'B'),
 ('D', 'F')]

In [135]:
get_game_intervals(get_matches(teams[:6]))

{'A': [0, 0, 0, 0, 0, 10],
 'B': [0, 4, 0, 0, 0, 6],
 'C': [1, 3, 3, 0, 0, 3],
 'D': [2, 3, 2, 2, 0, 1],
 'E': [3, 3, 2, 1, 1, 0],
 'F': [4, 3, 2, 1, 0, 0]}

In [137]:
get_game_intervals(get_priority_schedule(teams[:6]))

{'A': [0, 2, 1, 3, 1, 3],
 'B': [0, 3, 3, 3, 1, 0],
 'C': [1, 2, 2, 1, 3, 1],
 'D': [2, 2, 1, 2, 1, 2],
 'E': [2, 3, 1, 2, 1, 1],
 'F': [1, 1, 2, 3, 3, 0]}

In [102]:
def get_schedule(teams, max_retries=3):    
    all_matches = get_matches(teams)
    scheduled, remaining = all_matches[:1], all_matches[1:]
    next_remaining = 0
    while len(scheduled) < len(all_matches):
        while next_remaining < len(remaining):
            if not shares_team(remaining[next_remaining], scheduled[-1]):
                scheduled.append(remaining.pop(next_remaining))
                if remaining:
                    next_remaining = next_remaining % len(remaining)
                break
            next_remaining += 1
        if remaining and next_remaining >= len(remaining):
            scheduled.append(remaining.pop(0))
            next_remaining = 0
    scheduled = scheduled[::-1]  # prefer back to back games at beginning of tournament
    
    retries = 0
    while retries < max_retries and any_back_to_back(scheduled):
        scheduled = get_schedule(teams, max_retries=0)
    return scheduled

In [103]:
get_schedule(teams)

[('E', 'F'),
 ('B', 'G'),
 ('C', 'E'),
 ('D', 'G'),
 ('A', 'F'),
 ('C', 'D'),
 ('B', 'E'),
 ('A', 'C'),
 ('B', 'F'),
 ('E', 'H'),
 ('A', 'B'),
 ('D', 'H'),
 ('F', 'G'),
 ('A', 'D'),
 ('G', 'H'),
 ('D', 'E'),
 ('C', 'G'),
 ('D', 'F'),
 ('A', 'H'),
 ('B', 'C'),
 ('F', 'H'),
 ('E', 'G'),
 ('B', 'D'),
 ('C', 'F'),
 ('B', 'H'),
 ('A', 'E'),
 ('C', 'H'),
 ('A', 'G')]

In [104]:
get_schedule(teams[:5])

[('B', 'C'),
 ('A', 'E'),
 ('C', 'D'),
 ('B', 'E'),
 ('A', 'D'),
 ('C', 'E'),
 ('A', 'B'),
 ('D', 'E'),
 ('A', 'C'),
 ('B', 'D')]

In [105]:
get_schedule(teams[:6])

[('A', 'B'),
 ('E', 'F'),
 ('A', 'C'),
 ('B', 'F'),
 ('A', 'E'),
 ('B', 'D'),
 ('A', 'F'),
 ('B', 'C'),
 ('D', 'E'),
 ('C', 'F'),
 ('A', 'D'),
 ('C', 'E'),
 ('D', 'F'),
 ('B', 'E'),
 ('C', 'D')]